In [3]:
search_for = 125
start_from = 0
threads = 10
thread = 7
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.004260063171386719
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 35
all cases: 320327931
type: [1, 1, 1, 1, 125] 125
cases of this type: 244140625
100000 15.7021533222934
200000 26.873465933781834
300000 23.683035369096114
400000 27.005541849845056
500000 24.831179921545033
600000 26.79687187969819
700000 26.375766765627795
800000 23.529668255981793
900000 26.552774199545016
1000000 12.992188695933768
1100000 24.22823284003516
1200000 25.684343939610706
1300000 23.738933879537242
1400000 24.53667978255484
1500000 24.96590456304018
1600000 23.455125266694544
1700000 23.39673452104588
1800000 24.052473524305597
1900000 27.023208328545074
2000000 7.750266465033323
2100000 26.471089648012644
2200000 27.73502231129398
2300000 27.867707225488253
2400000 28.14743992412132
2500000 27.070855194403993
2600000 27.160877727243005
2700000 27.24028211393174
2800000 26.777526760836075
2900000 26.868943199258865
3000000 1

29400000 24.575135077446863
29500000 23.7259384323294
29600000 10.259058040518132
29700000 23.284771049678433
29800000 22.665341903362744
29900000 23.134966120588825
30000000 21.470792042823444
30100000 21.245516049695595
30200000 19.97751003191203
30300000 9.684960198420399
30400000 20.584347342325724
30500000 20.926007234774087
30600000 22.430576267021156
30700000 22.10614741587254
30800000 24.25807740619742
30900000 22.924481732236078
31000000 19.088659671315284
31100000 16.217927204116148
31200000 22.99727192567177
31300000 24.304041272263188
31400000 22.771099029867294
31500000 18.233435442473887
31600000 15.195920803983105
31700000 23.387564043142753
31800000 22.538756689208867
31900000 22.209190055815483
32000000 22.388582111620195
32100000 19.918650290704356
32200000 18.51208818364683
32300000 10.047402246571313
32400000 19.877041486562003
32500000 20.767923019541204
32600000 20.91798840749254
32700000 21.221913808573987
32800000 22.757048690082506
32900000 22.306560458727876
3

59100000 10.357921660928874
59200000 17.947657504039004
59300000 19.237387059862332
59400000 19.081031869021434
59500000 18.102523672484423
59600000 8.069387670673352
59700000 18.393503685891563
59800000 19.42704799386907
59900000 18.529760908810463
60000000 19.428584565721724
60100000 9.86354497086494
60200000 19.62369730234629
60300000 20.031493600193105
60400000 19.68570540304256
60500000 21.734752580920667
60600000 21.571830298192207
60700000 21.189366708311006
60800000 20.434026325144387
60900000 19.881952182044266
61000000 19.72613284210555
61100000 8.48718003110785
61200000 17.95755082657649
61300000 17.710020085996668
61400000 18.994696324807283
61500000 16.224358251116982
61600000 9.204121518590082
61700000 17.956110649820413
61800000 18.697258840729393
61900000 17.589633615138357
62000000 14.508367501570744
62100000 14.052439665898289
62200000 20.948180124604182
62300000 19.840862229374487
62400000 20.563576566566628
62500000 21.35781461993624
62600000 19.776128366679927
6270

88900000 6.149524513762743
89000000 15.970366463551722
89100000 15.562353016976784
89200000 9.371357481501963
89300000 15.849330110214956
89400000 16.713565504036104
89500000 17.030164685111345
89600000 17.311047935942618
89700000 17.34196903896309
89800000 17.73787231270855
89900000 17.792469494528913
90000000 17.79345667493988
90100000 16.356249241043724
90200000 16.321415658275505
90300000 16.390203559833044
90400000 15.597637171864726
90500000 15.236504615722204
90600000 8.173974612723693
90700000 14.178902606170174
90800000 13.427857591243649
90900000 8.179600084857398
91000000 14.887790877949353
91100000 8.901971134580682
91200000 15.86081137354889
91300000 16.42516063634032
91400000 15.304168152005683
91500000 16.885854953610274
91600000 16.573101996625805
91700000 17.762587274790924
91800000 16.995642077345384
91900000 18.0079313610478
92000000 17.679513030280877
92100000 17.401688109388775
92200000 16.30432778308843
92300000 16.673429942844752
92400000 15.923461449106583
92500

117900000 1.5491997161010722
118000000 1.6934655120590152
118100000 1.5597747865002694
118200000 1.5410414804304242
118300000 1.5011005300052966
118400000 1.7199062605851712
118500000 1.5842040838710647
118600000 1.5980570780401582
118700000 1.6243535117873642
118800000 1.5309361675925055
118900000 1.4992314678762018
119000000 1.5045146729541055
119100000 1.5928121192320108
119200000 1.528047228340635
119300000 1.5567723513082723
119400000 1.6112854494050861
119500000 1.6548860746370315
119600000 1.7006354067487095
119700000 1.5332432107895873
119800000 1.580509426676427
119900000 1.5946415541443002
120000000 1.7042209020383656
120100000 1.5050623120179143
120200000 1.5535164538966477
120300000 1.5953243853328187
120400000 1.5695796275060252
120500000 1.579657485450906
120600000 1.5419796961932477
120700000 1.5947340593833055
120800000 1.7076102068699974
120900000 1.4986452920168103
121000000 1.4946482300942825
121100000 1.5006384532498824
121200000 1.5804781634594183
121300000 1.84669

...

204300000 8.919275571075834
204400000 8.301147358132697
204500000 8.540024403900958
204600000 8.605074348616785
204700000 8.17718174967562
204800000 4.379214340659478
204900000 8.868166450778705
205000000 8.328397309888347
205100000 9.211761795923145
205200000 8.492236430248102
205300000 9.123614278855134
205400000 5.180437304111236
205500000 8.224084109680085
205600000 8.527031055964834
205700000 8.158579570075336
205800000 8.65061279894998
205900000 7.558556817609689
206000000 8.154648973233401
206100000 2.9760325030917545
206200000 7.755725437154926
206300000 8.30288123133205
206400000 7.868195404937179
206500000 8.539105274496798
206600000 8.308774636047223
206700000 7.970755750065112
206800000 5.0468094104404235
206900000 8.510058698424533
207000000 9.02768449084461
207100000 8.526201869516969
207200000 8.807753707394758
207300000 7.695182736052395
207400000 5.31605042520724
207500000 9.25514280343765
207600000 7.90829466508132
207700000 8.482755215395814
207800000 7.7188991225056

233600000 6.490690263888065
233700000 6.899609608949095
233800000 6.303680475030551
233900000 6.904291661056471
234000000 6.686241923701305
234100000 6.885333767095287
234200000 6.548987088789123
234300000 4.325838587266356
234400000 5.829900454219625
234500000 3.3607572331861295
234600000 7.031990080403725
234700000 6.279879478871859
234800000 6.586989392344072
234900000 6.720964837353031
235000000 6.152777102514963
235100000 6.48344709927404
235200000 6.2253333993387105
235300000 6.624631639771716
235400000 2.257303012195149
235500000 6.2829402995891215
235600000 6.8238577789054755
235700000 6.036971496757125
235800000 6.74874414301638
235900000 6.533392457639205
236000000 6.903540479067125
236100000 6.317600153726059
236200000 6.819997519700987
236300000 2.8035573558250415
236400000 4.1049008141726055
236500000 5.935384573959505
236600000 6.264971557182771
236700000 6.370489371077476
236800000 6.69488926357591
236900000 6.215621356575069
237000000 6.404630748817164
237100000 6.00391

262300000 1.6867194923664817
262400000 1.9231886485691765
262500000 2.0847757943273564
262600000 1.130433317482975
262700000 0.4850225630624447
262800000 0.48280755470510717
262900000 1.108686905594378
263000000 2.268887116547979
263100000 1.9032250364654304
263200000 1.7741402508873974
263300000 1.8111519390272788
263400000 1.6483506159962675
263500000 1.7812381822366712
263600000 2.04766240573424
263700000 2.0853289172317324
263800000 1.8234241625062233
263900000 1.7995707050410146
264000000 1.6940786624596
264100000 1.9457220332451117
264200000 1.5979583833224893
264300000 1.6865950409141162
264400000 1.8262539905420336
264500000 1.4565885846106574
264600000 0.7747503605263014
264700000 0.4718506573774854
264800000 0.7806065707230311
264900000 1.6476390016514124
265000000 2.0354332761097957
265100000 1.6764882030213994
265200000 1.5438215240035675
265300000 1.6638071648210322
265400000 1.96793777569391
265500000 1.6675391891629834
265600000 1.7202624118663292
265700000 1.97359302990

290800000 0.9839217725310716
290900000 0.933999103274529
291000000 1.194194694486314
291100000 1.0223855079456388
291200000 1.0278331365489097
291300000 0.8843312715680122
291400000 0.8917527386462377
291500000 1.132961687076999
291600000 0.8627916493375043
291700000 0.7942289496753692
291800000 1.0015623435644925
291900000 0.539885685498321
292000000 0.23917879766022962
292100000 0.23874237412441174
292200000 0.6451870573283016
292300000 1.0879600785974868
292400000 0.7630901077052374
292500000 0.900951342706575
292600000 0.9388256905785787
292700000 0.8988349379976127
292800000 1.0041253325980783
292900000 0.8906551934302097
best so far: 0
type: [1, 1, 1, 25, 5] 125
cases of this type: 9765625
293000000 0.9759465572036101
293100000 0.8151342483598154
293200000 0.8258088052522911
293300000 0.8563466998224311
293400000 0.7636069085984329
293500000 0.6634504756424892
293600000 0.5940808690877345
293700000 0.6213878612475813
293800000 0.7173410828108873
293900000 0.7440529576758961
29400

best so far: 0
type: [1, 1, 25, 5, 1] 125
cases of this type: 78125
317200000 0.019716309627178644
best so far: 0
type: [1, 1, 125, 1, 1] 125
cases of this type: 15625
best so far: 0
type: [1, 5, 1, 1, 25] 125
cases of this type: 1953125
317300000 0.017096132500980295
317400000 0.05408464630851944
317500000 0.05157243463370403
317600000 0.05105378191226721
317700000 0.04781661591163675
317800000 0.04474982801686393
317900000 0.034158290023015604
318000000 0.04046197496852279
318100000 0.03367490441377362
318200000 0.015314579720309708
318300000 0.013238022056194147
318400000 0.011260459321231313
318500000 0.016026018578243918
318600000 0.023705462081336972
318700000 0.025414806324336927
318800000 0.0287563814327187
318900000 0.02516531886267066
319000000 0.018925320462034146
319100000 0.018539423697356384
319200000 0.02196463553102016
best so far: 0
type: [1, 5, 1, 5, 5] 125
cases of this type: 390625
319300000 0.014142501806522741
319400000 0.007737778546684981
319500000 0.00620708865